In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np
from scipy.stats import norm
from gp_dev.core import *
from ddop.datasets import load_yaz
from pathlib import Path
import datetime
import category_encoders as ce
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, RationalQuadratic, ExpSineSquared

In [3]:
path = Path("..")
path_ds = path/'datasets'
path_res = path/'res_data'
path_plot = path/'plots'

In [ ]:
res = []
products = ['CALAMARI', 'FISH', 'SHRIMP', 'CHICKEN', 'KOEFTE', 'LAMB', 'STEAK']

for method in ['one-hot encoding', 'target encoding', ]:   
        
        if method == 'one-hot encoding':
            df = load_yaz(encode_date_features=True, include_prod=None, include_date=False).frame
        else:
            df = load_yaz(encode_date_features=False, include_prod=None, include_date=False).frame
        
        # make train/val/test
        n_train = 600
        train_df, test_df = df.iloc[:n_train, :], df.iloc[n_train:, :]
        n_test = len(test_df)
        
        train_df = pd.melt(train_df,
             id_vars = train_df.columns.difference(products),
             value_vars= products)

        test_df = pd.melt(test_df,
             id_vars = test_df.columns.difference(products),
             value_vars= products) 
        
        train_x_df, train_y_df = train_df.iloc[:, :-1], train_df.iloc[:, -1]
        test_x_df, test_y_df = test_df.iloc[:, :-1], test_df.iloc[:, -1]
        
        train_y = train_y_df.values
        test_y = test_y_df.values

        # target encoding
        if method == 'target encoding':
            for cat in ['WEEKDAY', 'MONTH', 'YEAR', 'ISHOLIDAY', 'WEEKEND', 'variable']:
                encoder = ce.TargetEncoder()
                train_x_df[cat] = encoder.fit_transform(train_x_df[cat].astype('category'), train_y_df)
                test_x_df[cat] = encoder.transform(test_x_df[cat].astype('category'))
        elif method == 'one-hot encoding':
            for cat in ['variable']:
                encoder = ce.OneHotEncoder()
                train_x_df = pd.concat([train_x_df, encoder.fit_transform(train_x_df[cat].astype('category'), train_y_df)], axis=1).drop(columns = cat)
                test_x_df = pd.concat([test_x_df, encoder.transform(test_x_df[cat].astype('category'))], axis=1).drop(columns = cat)
    
        scaler = MinMaxScaler()
        scaler.fit(train_x_df)

        train_x = scaler.transform(train_x_df)
        test_x = scaler.transform(test_x_df)

        # Kernel with parameters given in GPML book
        k1 = 1**2 * RBF(length_scale=0.261)  # long term smooth rising trend
        k2 = 2.4**2 * RBF(length_scale=90.0) \
            * ExpSineSquared(length_scale=1.3, periodicity=1.0)  # seasonal component
        k3 = 0.66**2 \
            * RationalQuadratic(length_scale=1.2, alpha=0.78) # medium term irregularity
        k4 = 0.18**2 * RBF(length_scale=0.134) \
            + WhiteKernel(noise_level=1.09**2)  # noise terms

        if method == 'timeseries':
            kernel_gpml = k1 + k2 + k3 + k4

        elif method == 'one-hot encoding':
            kernel_gpml = k1 + k4

        elif method == 'target encoding':
            kernel_gpml = k1 + k4

        gp = GaussianProcessRegressor(kernel=kernel_gpml, normalize_y=True)#, alpha=1)
        gp.fit(train_x, train_y)

        print("\nLearned kernel: %s" % gp.kernel_)
        print("Log-marginal-likelihood: %.3f"
              % gp.log_marginal_likelihood(gp.kernel_.theta))

        nv_means, y_std = gp.predict(test_x,  return_std=True)
        nv_sigma = y_std
        
        for i, target in enumerate(products):
            for c in range(5,100, 5):
                cu = c/100
                co = 1-cu

                nv_solution = nv_means[i*n_test:(i+1)*n_test]+norm.ppf(cu/(cu+co))*nv_sigma[i*n_test:(i+1)*n_test]
                cost =  np.mean([nv_cost(q, y, cu, co) for q, y in zip(nv_solution, test_y[i*n_test:(i+1)*n_test])])

                ser_tmp=pd.Series({"cu":cu, "co":co, "cost":cost, "type":method, "target": target, "split": 'test'})
                res.append(ser_tmp)
                
        nv_means, y_std = gp.predict(train_x,  return_std=True)
        nv_sigma = y_std
        
        for i, target in enumerate(products):
            for c in range(5,100, 5):
                cu = c/100
                co = 1-cu

                nv_solution = nv_means[i*n_train:(i+1)*n_train]+norm.ppf(cu/(cu+co))*nv_sigma[i*n_train:(i+1)*n_train]
                cost =  np.mean([nv_cost(q, y, cu, co) for q, y in zip(nv_solution, train_y[i*n_train:(i+1)*n_train])])

                ser_tmp=pd.Series({"cu":cu, "co":co, "cost":cost, "type":method, "target": target, "split": 'train'})
                res.append(ser_tmp)
            #df_res = pd.DataFrame(res)

for target in products:
    method = 'saa'
    df = load_yaz(encode_date_features=False, include_prod=[target], include_date=False).frame
    # make train/val/test
    n_train = 600
    train_df, test_df = df.iloc[:n_train, :], df.iloc[n_train:, :]

    train_x_df, train_y_df = train_df.iloc[:, :-1], train_df.iloc[:, -1]
    test_x_df, test_y_df = test_df.iloc[:, :-1], test_df.iloc[:, -1]

    train_y = train_y_df.values
    test_y = test_y_df.values

    for c in range(5,100, 5):
        cu = c/100
        co = 1-cu

        nv_quantile = np.quantile(train_y, q=cu/(cu+co))
        cost= np.mean([nv_cost(nv_quantile, y, cu, co) for y in test_y])
        nv_means, nv_sigma = 0,0

        ser_tmp=pd.Series({"cu":cu, "co":co, "cost":cost, "type":method, "target": target, "split": 'train'})
        res.append(ser_tmp)
        
df_res = pd.DataFrame(res)

/home/magr/anaconda3/envs/gp/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


nnn= 'SOF_results_Symmetric.csv'
df_tmp = pd.read_csv(nnn)
#df_tmp = df_tmp.drop(columns=["Unnamed: 0"])
df_tmp['target']="STEAK"
df_tmp.to_csv(nnn, index=False)

In [ ]:
df_plot = df_res
#df_plot = pd.read_csv('res_data/gp_all-paste.csv')
df_plot = df_plot[~(df_plot.type.isin(["rf_rf", "rf_grf", "rf_oracle"]))]
#df_plot = df_plot[~(df_plot.type.isin(["rf_approx_risk", "rf_approx_sol", "oracle"]))]
#df_plot = df_plot[~(df_plot.type.isin(["saa", "rf"]))]
sns.set(rc={'figure.figsize':(15,15)})
sns.set_style('whitegrid')
sns.relplot(data=df_plot, x="cu", y="cost",col_wrap=3,facet_kws={'sharey':False},
    col="target", hue="type",kind="line", aspect=1, height=4); 